# Install dependencies 

In [1]:
#! git clone https://github.com/ALjone/INF368-Final-Project

# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

!pip install -q tensorflow
!pip install -q tensorflow_hub

!pip install -q official.nlp


!pip install -q matplotlib
!pip install numpy
!pip install pandas

ERROR: Could not find a version that satisfies the requirement official.nlp (from versions: none)
ERROR: No matching distribution found for official.nlp


# Import libraries

In [2]:
import sys
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
os.chdir('..')
from Bert import Bert


# Set-up

Set gloabal variable for changing dataset.

Set data_name to the name of your dataset. This needs to correspond to a folder in /data/, which should be generated by the generate_data.ipynb notebook. num_classes manually needs to be set to the number of classes in your dataset.

- data_name: "imdb" or "medical"
- num_classes:
    - imdb: 2
    - medical: 5

In [3]:
## gloabal variable for changing dataset.
## data_name possible values: "imdb", "medical"
data_name = "medical"
## num_classes possible values: "imdb"=2, "medical"=5
num_classes = 5

## Get data paths
- Get the file path for training data sets, 5, 10, 25, and 50 per label
- Get the path for the test set

In [4]:
## get the path for the training data sets, 5, 10, 25, and 50 per label
data_path = 'data/'+data_name
files = os.listdir(data_path)
labeled_files = [data_path+"/"+file for file in files if "train_labeled" in file]

## get the path for the test set
test_path = data_path+"/test.csv"

## Set hyper-parameters for Bert

In [5]:
# hyper parameters 
batch_size = 4
seed = 0
learning_rate = 5e-5
epochs=5

# create a data frame to store the results
results=pd.DataFrame(columns=["n_per_class", "accuracy"])

# Traning and evalueate the Bert classifier

In [6]:
## train and evaluate bert for each data set. 
for n_per_class in [5,10,25,50]:
    data_file = ""
    result = {"n_per_class":n_per_class}
    bert = Bert(num_classes = num_classes, random_state = seed) # create model 
    for file in labeled_files: ## find correct file
        if f"train_labeled_{n_per_class}.csv" in file:
            data_file = file
            break
    print(data_file)
    ## train model
    bert.train_from_path(data_file,learning_rate=learning_rate,batch_size=batch_size,epochs=epochs)
    ## evaluate the model
    performance = bert.evaluate_from_path(test_path)
    ## add to resutl data frame
    result["accuracy"] = performance[1]
    results = results.append(result,ignore_index=True)


BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3


2021-11-11 19:36:54.988275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10800 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:06:00.0, compute capability: 3.7
2021-11-11 19:36:58.084455: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


data/medical/train_labeled_5.csv
Epoch 1/5
7/7 [==============================] - 22s 269ms/step - loss: 1.5570 - accuracy: 0.3600
Epoch 2/5
7/7 [==============================] - 2s 262ms/step - loss: 1.0369 - accuracy: 0.6000
Epoch 3/5
7/7 [==============================] - 2s 265ms/step - loss: 1.1334 - accuracy: 0.6000
Epoch 4/5
7/7 [==============================] - 2s 262ms/step - loss: 0.9176 - accuracy: 0.7600
Epoch 5/5
500/500 [==============================] - 18s 33ms/step - loss: 0.9731 - accuracy: 0.6000
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3
data/medical/train_labeled_10.csv
Epoch 1/5
13/13 [==============================] - 23s 278ms/step - loss: 2.0240 - accuracy: 0.1000
Epoch 2/5
13/13 [==============================] - 4s 272ms/step - loss: 1.4455 - accuracy: 0.4000
Epoch 3/5
13/13 [==============================] - 4s 273ms/ste

In [7]:
## print the result
results

,n_per_class,accuracy
0,5.0,0.600
1,10.0,0.274
2,25.0,0.562
3,50.0,0.520


In [8]:
# write the result to file
if not os.path.exists('results'):
      os.mkdir('results')
result_path = f'results/{data_name}'
if not os.path.exists(result_path):
      os.mkdir(result_path)
results.to_csv(f"{result_path}/bert_results.csv", index=False)